## Install and import bibraries

In [ ]:
# !pip install -qq datasets==2.16.1 evaluate==0.4.1 transformers[sentencepiece]==4.35.2
# !pip install -qq accelerate==0.26.1
# !apt install git-lfs

In [ ]:
import numpy as np
from tqdm.auto import tqdm
import collections

import torch

from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering
from transformers import TrainingArguments
from transformers import Trainer
import evaluate

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
# Sử dụng mô hình "distilbert-base-uncased"
# làm mô hình checkpoint
MODEL_NAME = "distilbert-base-uncased"

# Độ dài tối đa cho mỗi đoạn văn bản
# sau khi được xử lý
MAX_LENGTH = 384

# Khoảng cách giữa các điểm bắt đầu
# của các đoạn văn bản liên tiếp
STRIDE = 128

## Setup Dataset

### Load datasets

In [ ]:
DATASET_NAME = "rajpurkar/squad_v2"
raw_datasets = load_dataset(DATASET_NAME)

In [ ]:
raw_datasets

In [ ]:
raw_datasets['train'][0]

### Tokenizer

#### Lưu ý khi tokenize
truncation (Cắt bớt):

Mục đích: Quyết định có cắt bớt chuỗi đầu vào hay không, và nếu có thì cắt như thế nào. Hầu hết các mô hình transformer (như BERT, RoBERTa,...) có giới hạn độ dài đầu vào (ví dụ: 512 token). truncation giúp xử lý các chuỗi dài hơn giới hạn này.

Giá trị:

- False hoặc 'do_not_truncate' (mặc định): Không cắt bớt. Nếu chuỗi dài hơn giới hạn của mô hình, bạn có thể gặp lỗi.

- True hoặc 'longest_first': Cắt bớt theo nguyên tắc "dài nhất trước".

    - Nếu bạn đưa vào một chuỗi: Cắt chuỗi đó cho đến khi nó đạt độ dài tối đa (max_length hoặc độ dài tối đa của mô hình).

    - Nếu bạn đưa vào một cặp chuỗi (ví dụ: câu hỏi và ngữ cảnh): Cắt chuỗi dài hơn trong cặp cho đến khi tổng độ dài của cả hai chuỗi (cộng thêm các token đặc biệt như [CLS], [SEP]) đạt max_length. Tiếp tục cắt chuỗi dài hơn cho đến khi đạt được độ dài mong muốn.

- 'only_first': Chỉ cắt chuỗi thứ nhất trong cặp chuỗi (hoặc batch). Chuỗi thứ hai không bị ảnh hưởng.

- 'only_second': Chỉ cắt chuỗi thứ hai trong cặp chuỗi (hoặc batch). Chuỗi thứ nhất không bị ảnh hưởng.

max_length (Độ dài tối đa):

- Mục đích: Đặt giới hạn độ dài tuyệt đối cho chuỗi đầu ra (sau khi đã cắt bớt, nếu có).

- Giá trị:

    - int: Một số nguyên dương chỉ định độ dài tối đa.

    - None (mặc định): Nếu truncation được bật, max_length sẽ tự động được đặt bằng độ dài tối đa mà mô hình hỗ trợ (ví dụ: 512 với BERT). Nếu truncation tắt, max_length không có tác dụng.

- Quan trọng: max_length bao gồm cả các token đặc biệt ([CLS], [SEP], ...).

stride (Bước nhảy):

- Mục đích: Chỉ có tác dụng khi truncation=True và return_overflowing_tokens=True. Khi một chuỗi bị cắt bớt, stride cho phép bạn tạo ra các "cửa sổ trượt" (sliding windows) chồng lấn lên nhau, giúp giữ lại một phần ngữ cảnh từ phần bị cắt.

- Giá trị:
    - int : Số token được giữ lại (overlap) từ phần cuối của chuỗi đã cắt, và đưa vào đầu của chuỗi "overflowing" tiếp theo.

return_offsets_mapping là một tham số tùy chọn (mặc định là False) trong các hàm tokenizer của Hugging Face Transformers, nó quyết định xem có trả về thông tin về vị trí ký tự (character-level) của mỗi token trong chuỗi gốc hay không.
- Offsets: "Offsets" là một cặp số (char_start, char_end):
    - char_start: Vị trí ký tự bắt đầu của token trong chuỗi gốc.
    - char_end: Vị trí ký tự kết thúc của token trong chuỗi gốc (lưu ý: ký tự tại char_end không thuộc token). Nói cách khác, khoảng [char_start, char_end) chứa token.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
# Định nghĩa hàm preprocess_training_examples và nhận đối số examples là dữ liệu đào tạo
def preprocess_training_examples(examples):
    tokens = tokenizer(
        examples["question"], 
        examples["context"], 
        truncation="only_second", 
        max_length=MAX_LENGTH, 
        stride=STRIDE, 
        return_overflowing_tokens=True, 
        return_offsets_mapping=True, 
        padding="max_length")
    print(tokens)
    return None

In [ ]:
## Test tokenizer
text1 = "This is a very long sentence."  # Dài hơn max_length
text2 = "Short."

tokens = tokenizer(
    text1,
    text2,
    truncation="only_second",
    max_length=15,
    stride=2,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    padding="max_length")

print(tokens)
print(tokens.sequence_ids(0))
print(tokenizer.decode(tokens['input_ids'][0]))

In [ ]:
raw_datasets.map(preprocess_training_examples, batched=True)